# Deploying Machine Learning Models on GCP Kubernetes (with RBAC)

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
 - You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
 -  If you wish to test the JSON schema checks you will need presently to enbale "alpha features" for your cluster (Jan 2018).
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [1]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start Seldon-Core

In [2]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd \
    --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Sep  8 09:36:49 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                        READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-c4b6b4664-wqbgk  0/1    ContainerCreating  0         1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  1s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  1s


NOTES:
NOTES: TODO




In [3]:
!kubectl create namespace seldon

namespace "seldon" created


In [4]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --namespace seldon

NAME:   seldon-core
LAST DEPLOYED: Sat Sep  8 09:38:00 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME    AGE
seldon  0s

==> v1beta1/Role
NAME          AGE
seldon-local  0s

==> v1/RoleBinding
NAME    AGE
seldon  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-seldon-apiserver  NodePort   10.47.247.156  <none>       8080:32261/TCP,5000:31802/TCP  0s
seldon-core-redis             ClusterIP  10.47.255.152  <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis                   1        1        1           0          0s

==> v1/Pod(related)
NAME                                                 REA

## Set up REST and gRPC methods

**Ensure you port forward the seldon api-server REST and GRPC ports**:

REST:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

GRPC:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:5000
```

In [5]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [10]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8003"

## Normal Operation

### Create Seldon Deployment

In [6]:
!kubectl create -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [8]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1] map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1]]]

### Get Predictions

#### REST Request

In [11]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "p8ma5ucali82lf4m9t5vp2tptd",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.09380341852544855, 0.07219164170674934]
    }
  }
}


#### gRPC Request

In [12]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43196,"scope":"read write"}
meta {
  puid: "d0v8jbq2k33sa78sb3pkji3vh9"
}
data {
  names: "proba"
  tensor {
    shape: 2
    shape: 1
    values: 0.07365884585009366
    values: 0.06725779032488582
  }
}



## Update Deployment with Canary

In [13]:
!kubectl apply -f resources/model_with_canary.json -n seldon

seldondeployment "seldon-model" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [16]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:test-deployment-example-classifier-0] map[replicasAvailable:1 name:test-deployment-canary-svc-orch replicas:1] map[name:test-deployment-canary-mean-classifier-0 replicas:1 replicasAvailable:1]]]

In [17]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43131,"scope":"read write"}
{
  "meta": {
    "puid": "aicmssrcgakld9jtf8e9jdgbr7",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.07541984830478907, 0.08452793566856674]
    }
  }
}


In [18]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"7f97c128-aa37-4b76-b2cc-4eea99237f34","token_type":"bearer","expires_in":43128,"scope":"read write"}
meta {
  puid: "88q5lvu9rbe3jq4kpfki2l6gcf"
}
data {
  names: "proba"
  tensor {
    shape: 2
    shape: 1
    values: 0.08780586679952734
    values: 0.0825496801645638
  }
}



## Tear Down

In [ ]:
!kubectl delete -f resources/model.json -n seldon

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge